In [2]:
import os
import time
import numpy as np
import soundfile as sf
import librosa
import csv
import scipy.misc
import imageio
import shutil
import sys
import datetime
import random
import collections
import pickle
import math
from random import randint

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
# set the memory usage
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
set_session(tf.Session(config=tf_config))

from keras.layers import Dropout, AveragePooling2D,LSTM,Reshape, Bidirectional
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Input, BatchNormalization
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import losses, optimizers
#from keras.layers.pooling import GlobalMaxPooling1D

In [3]:
with open("datasets/train_5000_glassbreak_Prob1_44100_128_framelevel_calssifier.pkl", "rb") as input_file:
    train_data = pickle.load(input_file)
input_file.close()

In [4]:
train_frames = train_data['train_frames']
train_labels = train_data['train_labels']

In [5]:
len(train_frames)

5000

In [6]:
with open("datasets/val_1000_glassbreak_44100_128_framelevel_calssifier.pkl", "rb") as input_file:
    val_data = pickle.load(input_file)
input_file.close()

In [7]:
val_frames = val_data['val_frames']
val_labels = val_data['val_labels']

In [8]:
train_frames = np.array(train_frames).reshape(5000,128,200,1)
val_frames = np.array(val_frames[0:500]).reshape(500,128,200,1)
train_labels = np.array(train_labels).reshape(5000,200)
val_labels = np.array(val_labels[0:500]).reshape(500,200)

In [9]:
def get_1d_conv_model():
    
    inp = Input(shape=(128,200,1))
    
    x = Conv2D(64, (10,1), strides=(1,1) , padding="same")(inp)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((10,1),strides=(10,1))(x)
    x = Dropout(rate=0.1)(x)
    print(x._keras_shape)
    
    x = Conv2D(256, (7,1), strides=(1,1) , padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((7,1),strides=(7,1))(x)
    x = Dropout(rate=0.1)(x)
    print(x._keras_shape)
    
    
    x = Reshape((256,200))(x)
    
    x = LSTM(200, return_sequences=True, return_state=True)(x)
    
    x = LSTM(200, return_sequences=True)(x)
    
    
    print(x._keras_shape)
    x = Flatten()(x)
    out = Dense(200, activation='sigmoid')(x)
    
    model = Model(inputs=inp, outputs=out)
    opt = optimizers.Adam(lr=0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    return model

In [10]:
my_model = get_1d_conv_model()

(None, 12, 200, 64)
(None, 1, 200, 256)
(None, 256, 200)


In [11]:
my_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 200, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 200, 64) 704         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 200, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 128, 200, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [12]:
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=10, mode='auto')

In [13]:
my_model.fit(x=train_frames,y=train_labels,batch_size=150,epochs=100, 
             validation_data=(val_frames, val_labels), callbacks=[earlystop])

Train on 5000 samples, validate on 500 samples
Epoch 1/100
5000/5000 [==============================] - 58s 12ms/step - loss: 0.4746 - acc: 0.7809 - val_loss: 1.3650 - val_acc: 0.3443
Epoch 2/100
5000/5000 [==============================] - 56s 11ms/step - loss: 0.1639 - acc: 0.9353 - val_loss: 0.9566 - val_acc: 0.5839
Epoch 3/100
5000/5000 [==============================] - 56s 11ms/step - loss: 0.1262 - acc: 0.9485 - val_loss: 0.6695 - val_acc: 0.7543
Epoch 4/100
5000/5000 [==============================] - 56s 11ms/step - loss: 0.1118 - acc: 0.9556 - val_loss: 0.8346 - val_acc: 0.6939
Epoch 5/100
5000/5000 [==============================] - 56s 11ms/step - loss: 0.0990 - acc: 0.9602 - val_loss: 0.1241 - val_acc: 0.9498
Epoch 6/100
5000/5000 [==============================] - 56s 11ms/step - loss: 0.0903 - acc: 0.9639 - val_loss: 0.1443 - val_acc: 0.9406
Epoch 7/100
5000/5000 [==============================] - 56s 11ms/step - loss: 0.0824 - acc: 0.9669 - val_loss: 0.4070 - val_acc: 0

Epoch 60/100
5000/5000 [==============================] - 56s 11ms/step - loss: 3.6125e-04 - acc: 1.0000 - val_loss: 0.0975 - val_acc: 0.9803
Epoch 61/100
5000/5000 [==============================] - 56s 11ms/step - loss: 3.7049e-04 - acc: 0.9999 - val_loss: 0.1027 - val_acc: 0.9797
Epoch 62/100
5000/5000 [==============================] - 56s 11ms/step - loss: 3.4150e-04 - acc: 1.0000 - val_loss: 0.1000 - val_acc: 0.9801
Epoch 63/100
5000/5000 [==============================] - 56s 11ms/step - loss: 2.4619e-04 - acc: 1.0000 - val_loss: 0.0993 - val_acc: 0.9801
Epoch 64/100
5000/5000 [==============================] - 56s 11ms/step - loss: 1.9587e-04 - acc: 1.0000 - val_loss: 0.1012 - val_acc: 0.9800
Epoch 65/100
5000/5000 [==============================] - 56s 11ms/step - loss: 1.8884e-04 - acc: 1.0000 - val_loss: 0.1013 - val_acc: 0.9804
Epoch 66/100
5000/5000 [==============================] - 56s 11ms/step - loss: 1.8438e-04 - acc: 1.0000 - val_loss: 0.1031 - val_acc: 0.9801
Epoch 

In [22]:
my_model = load_model('final_melspec_glassbreak_model_2cnn_64_256_2lstm_1dense_200_04.h5')

In [4]:
with open('test_set_2_500_glassbreak_44100_128_multiple_estimated_12_preprocessed_with_confidence0.6.txt','r') as regions_file:
    reader = (regions_file.read())
    l = list(reader.split('\n'))
regions_file.close()

In [5]:
l[0]

'mixture_devtrain_glassbreak_000_550a91b27402aa2427f3ab0a465a1334.wav;10.482758620689655;12.068965517241379;glassbreak;0.980227'

In [6]:
#my_model.save('final_melspec_glassbreak_model_2cnn_64_256_2lstm_1dense_200_06.h5')

In [7]:
frames_per_sample = 200

def create_melspec_test(path,start_time,finish_time):
    test_frames=[]
    
    melspec = imageio.imread(path)
    print(path)
    event_start_index = int(np.round(float(start_time)/30 * 1305))
    event_finish_index = int(np.round(float(finish_time)/30 * 1305))
    event_length = event_finish_index - event_start_index 
    print(event_start_index)
    print(event_finish_index)
                    
    #calculating sample start and finish indices, randomly
    sample_start = randint(max(0,event_start_index - max(0,(200-event_length))),event_start_index)
    sample_finish = min(1305,sample_start+frames_per_sample)
                    
    print('before')
    print(sample_start)
    print(sample_finish)
    sample_length = sample_finish - sample_start 
    if sample_length < frames_per_sample:#if we don't have enough negative samples after event
        sample_start = sample_start - (frames_per_sample - sample_length)
                    
    print('after')
    print(sample_start)
    print(sample_finish)
                    
    temp1 = np.array(melspec)
    temp_test_frames = temp1[:,sample_start:sample_finish]
                    
    print(np.array(temp_test_frames).shape)
    
    temp2 = np.array(temp_test_frames)
    temp2 = temp2.reshape(128,200)
    test_frames = temp2
                    
    print(np.array(test_frames).shape)
    
    
    return test_frames

In [8]:
def choose_most_probable_event(dic):
    delimeter = '\t'
    ind = 0
    max_val = 0
    max_above1 = 0
    
    for i in range(len(dic['predicted_values'])):
        temp_list = dic['predicted_values'][i][0]
        conf = dic['maskrcnn_conf'][i]
        #print(temp_list)
        print('conf is '+str(conf))
        
        '''
        max_val_one_area = max(temp_list)
        
        if max_val_one_area > max_val:
            max_val = max_val_one_area
            ind = i
        
        average_one_area = float(sum(temp_list)/12)
        print(average_one_area)
        if average_one_area > max_val:
            max_val = average_one_area
            ind = i
        '''
        m = max(temp_list)
        sum_above1 = 0
        num_above1 = 0
        for j in temp_list:
            if j>=0.1:
                sum_above1 += j
                num_above1 += 1
        if num_above1 > 0:
            avg_above1 = float(sum_above1/num_above1)
        else:
            avg_above1 =0 
            
        if (0.7*(avg_above1) + 0.3*(conf)) > max_val:
            max_val = 0.7*(avg_above1) + 0.3*(conf)
            max_above1 = avg_above1
            max_m = m
            ind = i
        print('avg above 1 is '+str(avg_above1))
        #print('0.7*(avg_above1) + 0.3*(conf) is '+str(0.7*(avg_above1) + 0.3*(conf)))
             
        
    if max_val >= 0.8:
        st = dic['file_name'][ind]+ delimeter + str(dic['event_start_time'][ind]) + delimeter + str(dic['event_end_time'][ind]) + delimeter + event
    else:
        st = dic['file_name'][ind]+ delimeter + '0.0\t0.0\tno_event'
    return st
    

In [9]:
logs_path = 'logs/'
log_path = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
log_path = os.path.join(logs_path, log_path)

if os.path.exists(log_path):
    pass
else:
    os.mkdir(log_path)
shutil.copyfile('LogMel_Classifier-with_context_glassbreak.ipynb', os.path.join(log_path, 'logmel_classifier_code.ipynb'))
 

'logs/2018-11-30-16-43/logmel_classifier_code.ipynb'

In [25]:
event = 'glassbreak'
test_path = '../Maskrcnn_audio/datasets/test_set_2_glassbreak/'
previous_file_name = l[0].split(';')[0].replace('[','').replace("'",'')
dic = collections.defaultdict(list)


#with open(os.path.join(log_path+'/final_report.txt'),'w') as w_file:
with open(os.path.join(log_path + '/final_report.txt'),'w') as w_file:
    for element in l:
        parts = element.split(';')
        
        if(parts[3]==event):
            current_file_name = parts[0].replace('[','').replace("'",'')
            start_time = float(parts[1])
            finish_time = float(parts[2])
            
            
            test_frames = create_melspec_test(test_path + current_file_name +'.png', start_time, finish_time)
            test_frames = np.array(test_frames)
            test_frames = test_frames.reshape(1,128,200,1)
            print(test_frames.shape)
            
            predictions = my_model.predict(x=test_frames)
            
            print(parts[0]+ parts[1]+parts[2])
            print(predictions)
                    
            predict_values = np.array(predictions)
            if current_file_name == previous_file_name:
                dic['file_name'].append(current_file_name)
                dic['event_start_time'].append(float(parts[1]))
                dic['event_end_time'].append(float(parts[2]))
                dic['predicted_values'].append(predict_values)
                dic['maskrcnn_conf'].append(float(parts[4]))
            else:
                
                st = choose_most_probable_event(dic)
                w_file.write(st+'\n')
                dic.clear()
                previous_file_name = current_file_name
                dic['file_name'].append(current_file_name)
                dic['event_start_time'].append(float(parts[1]))
                dic['event_end_time'].append(float(parts[2]))
                dic['predicted_values'].append(predict_values)
                dic['maskrcnn_conf'].append(float(parts[4]))
        else:
            w_file.write(element+'\n')
    #for the last file
    st = choose_most_probable_event(dic)
    w_file.write(st+'\n')
    dic.clear()    
            
w_file.close()

../Maskrcnn_audio/datasets/test_set_2_glassbreak/mixture_devtrain_glassbreak_000_550a91b27402aa2427f3ab0a465a1334.wav.png
456
525
before
396
596
after
396
596
(128, 200)
(128, 200)
(1, 128, 200, 1)
mixture_devtrain_glassbreak_000_550a91b27402aa2427f3ab0a465a1334.wav10.48275862068965512.068965517241379
[[1.52236961e-13 1.84120514e-11 2.37414628e-14 2.81750453e-13
  7.73574756e-16 5.72838055e-13 7.99168110e-12 1.60662160e-13
  6.00114161e-16 8.58185571e-12 2.73573509e-13 1.13961189e-15
  2.28371862e-16 2.44228228e-15 2.74384650e-16 6.27621306e-12
  1.59467307e-12 4.61507221e-11 1.53573126e-11 4.90537989e-14
  9.67498542e-13 9.25840681e-16 4.36164539e-14 2.46808328e-16
  5.58065646e-16 2.83944469e-17 4.82152160e-15 8.85369523e-13
  2.33350860e-17 6.83409144e-17 9.20929660e-14 4.60964023e-09
  1.26067234e-12 8.30178027e-12 5.84475339e-16 6.92394758e-15
  3.15514390e-12 1.69057357e-12 1.13334906e-12 6.89899730e-14
  1.40292949e-14 4.20440158e-12 3.62915642e-10 1.15055738e-13
  1.45258350e-0

mixture_devtrain_glassbreak_004_73d63cc0a537f18416a30b0a036d7436.wav3.56321839080459765.908045977011494
[[1.32933855e-11 1.37959932e-14 1.41068743e-13 3.72139625e-10
  2.42101223e-15 2.97016751e-13 2.58546032e-11 1.31530793e-11
  9.34931021e-10 3.21070115e-10 6.73562262e-10 1.73472031e-07
  2.51005581e-08 1.76813235e-04 7.72411681e-08 1.32344125e-04
  6.79499507e-02 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 9.99999642e-01 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e

mixture_devtrain_glassbreak_006_d409e08c3c1b3c944664e1549d120446.wav22.9655172413793125.42528735632184
[[8.79720288e-13 6.44922440e-12 1.56196073e-15 6.15680087e-13
  2.13162235e-08 3.76797871e-10 2.94928873e-11 2.87111956e-09
  7.50270885e-08 2.96492525e-10 1.75291639e-08 1.62692789e-13
  2.15393187e-13 2.13708634e-11 8.32066915e-10 9.70676339e-08
  3.48418572e-10 7.45457953e-07 7.68672415e-09 4.01511047e-10
  4.46684273e-10 8.01298047e-14 1.80213799e-08 2.45915891e-13
  4.68707375e-13 1.58982147e-10 1.73257184e-14 1.14984665e-07
  5.47165113e-09 1.30998963e-08 5.26935695e-09 3.45519631e-08
  4.67142858e-09 2.48490437e-08 1.88760430e-09 1.77732190e-07
  5.71389265e-08 7.45677767e-07 9.75132763e-01 9.99996185e-01
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 9.99999881e-01
  1.00000000e+00 1.00000000e+00 9.99963045e-01 1.00000000e+00
  1.00000000e+00 1.00000000e+

mixture_devtrain_glassbreak_014_66e90979fa495693e9336f2d98552533.wav17.7471264367816118.804597701149426
[[3.07692455e-10 2.66176034e-17 7.39243174e-17 7.14794788e-19
  2.35023064e-18 2.40590180e-17 3.59139455e-17 8.51333614e-19
  1.51719152e-18 2.05423770e-17 1.68033098e-20 4.91421012e-18
  1.21761158e-16 2.40305540e-12 3.68686418e-13 8.65880371e-12
  7.03308463e-17 1.51896962e-14 7.18374572e-15 6.50040508e-11
  9.96732510e-08 4.81568022e-05 2.49843113e-03 1.34607806e-04
  1.09836674e-05 5.79809182e-07 9.97792540e-07 5.28214400e-07
  4.84199791e-06 6.23584992e-06 1.52346934e-06 5.96478333e-09
  7.91770731e-07 4.83337999e-07 2.13374762e-10 7.96984645e-10
  9.43047238e-11 6.49181819e-11 6.91701904e-11 2.99480644e-12
  8.05600465e-12 4.54216607e-13 4.35969533e-12 6.85739931e-13
  9.15612746e-13 6.75801044e-15 2.32997324e-13 1.58839765e-13
  2.23774532e-09 1.07406075e-13 8.04857194e-08 8.96589747e-11
  2.49760324e-10 2.00252162e-10 4.31165752e-08 9.71652625e-08
  5.16855501e-08 1.27216226e

mixture_devtrain_glassbreak_017_f8b32fe70c24000f3e60a3ab624e12fa.wav23.4942528735632224.597701149425287
[[9.12654743e-11 3.16665479e-11 1.22253552e-09 6.62115219e-13
  1.58368124e-13 7.46863441e-14 1.69669661e-14 6.04046306e-08
  6.59969605e-12 2.79812597e-11 6.17946319e-12 2.66680349e-08
  8.52586682e-11 5.14184875e-12 3.83790805e-10 8.17915017e-13
  3.34905693e-12 9.80647922e-14 7.91325826e-13 1.04928422e-09
  6.66434130e-10 1.72399851e-07 7.24465821e-08 8.75001316e-09
  3.01805780e-09 1.44858339e-08 1.95492653e-08 1.59916351e-11
  1.36320082e-07 5.79821879e-08 1.09606123e-10 6.07293599e-12
  6.95436347e-11 6.15528115e-11 1.40947742e-09 1.36140220e-12
  2.95526902e-11 8.60136886e-13 3.30885374e-15 5.68375416e-11
  2.42443435e-13 3.82426903e-12 2.37482343e-12 4.34622219e-15
  5.69414036e-14 5.83614277e-12 3.48456299e-11 1.50151086e-10
  5.38755238e-12 6.53992693e-11 4.39184696e-11 6.78066674e-12
  4.05812726e-13 4.46513443e-12 1.83757835e-14 2.85760565e-11
  6.79981280e-11 7.32205180e

...

mixture_devtrain_glassbreak_493_88266571eba906e7d21514645bebaafd.wav1.51724137931034482.3908045977011496
[[4.15312627e-07 3.95767273e-11 8.35875640e-12 1.04022535e-09
  4.29030393e-08 1.63094364e-11 8.56306581e-09 1.54637689e-08
  2.67607743e-06 1.40637302e-09 1.61003371e-08 6.42274012e-10
  7.06483192e-08 1.10105292e-09 9.58978452e-11 1.76149975e-08
  2.17822169e-08 4.31229182e-07 2.77823091e-07 5.22440416e-04
  9.78507280e-01 9.99999881e-01 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 9.99999642e-01 9.99994636e-01 1.00000000e+00
  9.99999881e-01 9.99997973e-01 9.99999762e-01 9.99999642e-01
  1.00000000e+00 9.99999762e-01 9.99993682e-01 1.00000000e+00
  9.99999881e-01 1.00000000e+00 9.99958992e-01 9.99981523e-01
  9.99917984e-01 9.97764707e-01 9.99732792e-01 9.97003496e-01
  1.95737749e-01 8.85760963

mixture_devtrain_glassbreak_496_27e77a08d89fcaea20c1f35bffae5be7.wav10.11494252873563210.528735632183908
[[1.12179743e-09 1.14577892e-09 6.30143671e-09 3.20885590e-07
  2.90779269e-07 9.86084672e-08 9.51553466e-06 1.55604653e-06
  2.26038077e-10 1.00518012e-08 1.01050823e-09 2.40440477e-06
  2.58771962e-07 2.91154436e-07 3.48481772e-05 7.50916627e-08
  1.04672608e-06 4.81358143e-07 2.85446708e-06 2.30530022e-05
  1.02659083e-04 2.94855836e-05 7.84387396e-07 9.96744056e-05
  8.57904112e-08 1.25215138e-07 3.95824529e-08 3.20094529e-08
  6.67167285e-07 6.77327307e-07 6.14363174e-08 1.39663371e-06
  3.43384414e-08 6.27053041e-08 3.38684686e-06 3.11708573e-06
  2.39274954e-07 7.72679840e-08 2.55413433e-08 8.01179012e-10
  1.05442686e-08 8.85136497e-09 1.21458361e-05 2.07894457e-09
  3.20638481e-11 4.43136692e-08 1.05898135e-09 4.97486843e-11
  8.97177799e-08 4.60670613e-08 9.05946490e-06 9.68854437e-08
  1.19320219e-07 1.92268104e-08 4.29900346e-08 2.48912670e-07
  4.47255957e-07 1.81144255

mixture_devtrain_glassbreak_497_935f4fe5db058686cc85612a4347c330.wav4.5517241379310358.367816091954023
[[6.07612150e-16 5.95669347e-11 2.71229500e-15 1.21111354e-14
  4.74193201e-14 6.99477767e-11 4.63524293e-13 3.54191638e-14
  5.42340803e-13 2.54104463e-11 7.68088704e-10 1.09175845e-11
  4.39567192e-13 1.70725604e-11 2.56809873e-13 6.00468026e-12
  2.79794688e-13 1.59997189e-11 4.36646119e-12 2.35524924e-13
  3.83943675e-11 5.56441171e-10 3.34436095e-06 1.97729810e-09
  5.26835658e-11 1.17931401e-10 2.82338875e-10 2.86093371e-10
  6.70899603e-09 7.46777175e-07 3.13741481e-07 1.28650279e-09
  1.95851113e-10 2.71718818e-07 5.08003723e-05 2.06720546e-01
  1.02427639e-01 9.22878981e-01 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 9.99999881e-01 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+

mixture_devtrain_glassbreak_499_3cfb7b5e6a8da950a37b413caec89af5.wav0.229885057471264360.5747126436781609
[[8.69465944e-10 1.63495137e-08 3.75216089e-08 1.74539147e-10
  1.70450667e-11 4.09222842e-08 1.67963378e-07 3.96800033e-07
  1.90836191e-09 2.49703647e-10 4.33526520e-06 1.82202764e-09
  2.71652220e-06 6.51167866e-05 1.95236920e-04 9.99071956e-01
  9.99990821e-01 9.99999762e-01 9.99999762e-01 9.99999762e-01
  1.00000000e+00 9.99999642e-01 1.00000000e+00 1.00000000e+00
  9.99999404e-01 9.99997020e-01 9.99999642e-01 9.99919534e-01
  4.55467105e-01 6.85492754e-01 5.25220335e-02 1.04280817e-03
  1.38656246e-02 2.26107216e-03 3.66724998e-04 1.61425248e-02
  6.16471953e-05 1.28082966e-03 8.11081007e-03 1.16747501e-03
  1.01198520e-05 2.69000680e-04 6.77493347e-07 3.36754979e-08
  8.97954600e-09 1.40921514e-07 1.17938362e-06 7.73210207e-09
  2.43806953e-05 1.00956333e-03 1.50949302e-06 2.98054914e-08
  6.05673867e-09 5.63775338e-09 5.03829867e-09 3.49569040e-09
  4.70795092e-09 4.1729186

mixture_devtrain_glassbreak_500_ed1f06fa5bd1d6eaf090381b256fa919.wav14.91954022988505616.988505747126435
[[3.48146872e-14 4.99516574e-12 2.33439922e-13 1.84689422e-13
  5.08908779e-14 8.43547745e-15 9.80673740e-15 6.79717894e-15
  6.09548626e-15 1.08021025e-12 2.57361373e-14 3.68896878e-16
  9.75054963e-17 1.45925698e-12 1.88224612e-14 1.33796680e-11
  8.37088916e-14 4.26902350e-12 1.40437512e-14 1.19138057e-13
  4.51918071e-16 9.92868727e-17 3.46235388e-12 8.52098882e-15
  3.63805462e-14 2.99412933e-14 1.41030894e-12 5.53666878e-12
  6.02830437e-17 2.40281276e-12 1.17496663e-11 2.93857508e-11
  1.95638210e-11 2.01450930e-13 1.39252707e-10 6.95537811e-13
  1.36141975e-09 2.07283704e-11 1.55887889e-13 1.20151446e-14
  2.78881496e-09 3.75191996e-11 1.23614591e-14 7.36471837e-14
  1.95132688e-13 2.07361564e-15 1.75053555e-14 4.31225151e-15
  9.29520589e-17 2.17920501e-17 1.18464774e-14 2.78665184e-16
  9.32191861e-15 1.09833178e-14 4.11760774e-14 9.66626518e-12
  2.07190103e-14 2.86562495

In [9]:
import math

In [10]:
def check_event_truth(target_name,start_time):
    with open('event_list_evaltest_gunshot.csv', mode='r') as csv_file:
        reader = list(csv.reader(csv_file, delimiter='\n'))
        
        for row,i in zip(reader,range(500)):
            fields= str(row).split('\\t')
            if len(fields)>1:
                
                if fields[0].replace('[','').replace("'",'') == target_name and fields[3].replace(']','').replace("'",'')==event and abs(float(fields[1])-float(start_time))<=0.5:
                    return True
    
    return False

In [11]:
#gets a file name and return the number in it!
def get_number_from_filename(s):
    p = s.split('_')
    return p[3]

In [22]:
#creating file with probability feature values
event = 'gunshot'
test_path = '../Maskrcnn_audio/datasets/eval_500_gunshot_44100_128/'
with open('eval_regions_melspec_44100_128_37_models_features_3cnn_1bidirectionLSTM_64.csv','w') as w_file:
    w_writer = csv.writer(w_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    w_writer.writerow(['file_name', 'start', 'finish','min_val', 'max_val','average','average of probs>=0.9',
                       'average of probs>=0.8','average of probs>=0.7','average of probs>=0.5','average of probs>=0.1'
                       ,'average over all','True/False','Mask-rcnn confidence','prob>0.9+conf','0.7*prob9+0.3conf',
                       '0.3*prob9+0.7conf','0.7*prob1+0.3*conf',
                               '0.3*prob1+0.7*conf'])
    
    for element in l:
        parts = element.split(';')
        
        if(parts[3]==event):
            current_file_name = parts[0].replace('[','').replace("'",'')
            start_time = float(parts[1])
            finish_time = float(parts[2])
            
            
            test_frames = create_melspec_test(test_path + current_file_name +'.png', start_time, finish_time)
            
            
            test_frames = test_frames.reshape(1,128,200,1)
            print(test_frames.shape)
            
            
            predict_values = []
            predictions = my_model.predict(x=test_frames)
            
            
            print(parts[0]+ parts[1]+parts[2])
            
            predict_values = predictions.tolist()
            
            file_number = get_number_from_filename(current_file_name)
            min_val = str(min(predict_values[0]))
            max_val = str(max(predict_values[0]))
            avg_val = str(sum(predict_values[0])/12)
            sum_above_9 = 0
            num_of_probs_above_9 = 0
            sum_above_8 = 0
            num_of_probs_above_8 = 0
            sum_above_7 = 0
            num_of_probs_above_7 = 0
            sum_above_5 = 0
            num_of_probs_above_5 = 0
            sum_above_1 = 0
            num_of_probs_above_1 = 0
            sum_over_all = 0
            avg_over_all = 0
            
            for i in range(len(predict_values[0])):
                sum_over_all += predict_values[0][i]
                if predict_values[0][i]>= 0.9:
                    sum_above_9 += predict_values[0][i]
                    num_of_probs_above_9 += 1
                if predict_values[0][i]>= 0.8:
                    sum_above_8 += predict_values[0][i]
                    num_of_probs_above_8 += 1
                if predict_values[0][i]>= 0.7:
                    sum_above_7 += predict_values[0][i]
                    num_of_probs_above_7 += 1
                if predict_values[0][i]>= 0.5:
                    sum_above_5 += predict_values[0][i]
                    num_of_probs_above_5 += 1
                if predict_values[0][i]>= 0.1:
                    sum_above_1 += predict_values[0][i]
                    num_of_probs_above_1 += 1
                   
            if num_of_probs_above_9 > 0:
                avg_above_9 = float(sum_above_9 / num_of_probs_above_9)
            else:
                avg_above_9 = 0
            if num_of_probs_above_8 > 0:
                avg_above_8 = float(sum_above_8 / num_of_probs_above_8)
            else:
                avg_above_8 = 0
            if num_of_probs_above_7 > 0:
                avg_above_7 = float(sum_above_7 / num_of_probs_above_7)
            else:
                avg_above_7 = 0
            if num_of_probs_above_5 > 0:
                avg_above_5 = float(sum_above_5 / num_of_probs_above_5)
            else:
                avg_above_5 = 0
            if num_of_probs_above_1 > 0:
                avg_above_1 = float(sum_above_1 / num_of_probs_above_1)
            else:
                avg_above_1 = 0
            avg_over_all = float(sum_over_all/len(predict_values[0]))
            
            region_truth = check_event_truth(current_file_name,parts[1])
            w_writer.writerow([file_number,parts[1],parts[2] , min_val , max_val , avg_val,avg_above_9, 
                               avg_above_8, avg_above_7, avg_above_5,avg_above_1,avg_over_all,region_truth,parts[4],
                               avg_above_9+float(parts[4]),0.7*avg_above_9+(0.3*float(parts[4])),
                               0.3*avg_above_9+(0.7*float(parts[4])),0.7*avg_above_1+(0.3*float(parts[4])),
                               0.3*avg_above_1+(0.7*float(parts[4]))])
            
            
            
        else:
            pass
            
            
w_file.close()